In [1]:
# base imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

In [2]:
# torch imports
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision import transforms

In [3]:
# Custom Imports
from src.CustomUTK import UTKDataset
from src.MultNN import TridentNN

In [4]:
# Read in Unzipped file and make into a gzip
# DONT NEED TO RUN THIS AGAIN
# dataFrame = pd.read_csv('./data/age_gender.csv')
# dataFrame.to_csv('./data/age_gender.gz', compression='gzip')

## Reading in Data

In [5]:
# Read in the dataframe
dataFrame = pd.read_csv('./data/age_gender.gz', compression='gzip')

# Split into training and testing
train_dataFrame, test_dataFrame = train_test_split(dataFrame, test_size=0.2)

# get the number of unique classes for each group
'''
class_nums = {'age_num':len(dataFrame['age'].unique()), 'eth_num':len(dataFrame['ethnicity'].unique()),
              'gen_num':len(dataFrame['gender'].unique())}
'''
class_nums = {'age_num':1, 'eth_num':len(dataFrame['ethnicity'].unique()),
              'gen_num':len(dataFrame['gender'].unique())}

# Define train and test transforms
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.49,), (0.23,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.49,), (0.23,))
])

# Construct the custom pytorch datasets
train_set = UTKDataset(train_dataFrame, transform=train_transform)
test_set = UTKDataset(test_dataFrame, transform=test_transform)

# Load the datasets into dataloaders
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False)

In [6]:
# Sanity Check
for X, y in train_loader:
    print(f'Shape of training X: {X.shape}')
    print(f'Shape of y: {y.shape}')
    break

Shape of training X: torch.Size([64, 1, 48, 48])
Shape of y: torch.Size([64, 3])


<br>
<br>
<br>

## Testing the datatypes

Printing out the shape and datatypes of the labels

In [7]:
for X, y in train_loader:
    age, gen, eth = y[:,0].resize_(len(y[:,0]),1).float(), y[:,1], y[:,2]
    print(f'Shape of age: {age.shape}  datatype of age: {age.dtype}')
    print(f'Shape of gen: {gen.shape}  datatype of age: {gen.dtype}')
    print(f'Shape of eth: {eth.shape}  datatype of age: {eth.dtype}')
    break

Shape of age: torch.Size([64, 1])  datatype of age: torch.float32
Shape of gen: torch.Size([64])  datatype of age: torch.int64
Shape of eth: torch.Size([64])  datatype of age: torch.int64


Age has to be a float, testing out how to make a tensor of float

In [8]:
sample_age = np.array(dataFrame.age[:])
sample_age_tensor = torch.tensor(sample_age)
print(sample_age.shape)
print(sample_age_tensor.shape)
print(sample_age_tensor.dtype)
sample_age_tensor = sample_age_tensor.float()
print(sample_age_tensor.dtype)

(23705,)
torch.Size([23705])
torch.int64
torch.float32


<br>
<br>

Printing out the shape and datatypes of the outputs of the model

In [7]:
class_nums = {'age_num':1, 'eth_num':len(dataFrame['ethnicity'].unique()),
              'gen_num':len(dataFrame['gender'].unique())}

tridentNN = TridentNN(class_nums['age_num'], class_nums['gen_num'], class_nums['eth_num'])

In [10]:
iterator = iter(train_loader)
x_batch, y_batch = iterator.next()

In [11]:
print(x_batch.shape)
print(y_batch.shape)

torch.Size([64, 1, 48, 48])
torch.Size([64, 3])


In [12]:
pred = tridentNN(x_batch)

In [13]:
age_pred, gen_pred, eth_pred = pred[0], pred[1], pred[2]
print(f'Shape of age: {age_pred.shape}  datatype of age: {age_pred.dtype}')
print(f'Shape of gen: {gen_pred.shape}  datatype of age: {gen_pred.dtype}')
print(f'Shape of eth: {eth_pred.shape}  datatype of age: {eth_pred.dtype}')

Shape of age: torch.Size([64, 1])  datatype of age: torch.float32
Shape of gen: torch.Size([64, 2])  datatype of age: torch.float32
Shape of eth: torch.Size([64, 5])  datatype of age: torch.float32


In [14]:
print(F.softmax(gen_pred[0], dim=0))

tensor([0.4928, 0.5072], grad_fn=<SoftmaxBackward>)


Everything looks good, lets just find a nice way to change the label size for age

In [15]:
print(sample_age_tensor.shape)
sample_age_tensor.resize_(len(sample_age_tensor),1)
print(sample_age_tensor.shape)
print(sample_age_tensor.dtype)

torch.Size([23705])
torch.Size([23705, 1])
torch.float32


<br>
<br>

## Testing out one forward and backward pass

In [8]:
# Define loss functions
age_loss = nn.MSELoss()
gen_loss = nn.CrossEntropyLoss() # TODO : Explore using Binary Cross Entropy Loss?
eth_loss = nn.CrossEntropyLoss()

# Define optimizer
opt = torch.optim.Adam(tridentNN.parameters(), lr=0.001)

In [9]:
# Get one sample from train loader and unpack y to get true labels
iterator = iter(train_loader)
X, y = iterator.next()

age, gen, eth = y[:,0].resize_(len(y[:,0]),1).float(), y[:,1], y[:,2]

### Forward Pass

In [10]:
# Get model prediction
pred = tridentNN(X)

In [11]:
# Calculate Loss
loss = age_loss(pred[0],age) + gen_loss(pred[1],gen) + eth_loss(pred[2],eth)

print(loss)

tensor(504.9414, grad_fn=<AddBackward0>)


In [12]:
# Backpropagate
# Backpropagation
opt.zero_grad()          # Zero the gradient
loss.backward()          # Calculate updates

# Gradient Descent
opt.step()         # Apply updates

In [14]:
print(loss)

tensor(504.9414, grad_fn=<AddBackward0>)


## Find way to vectorize accuarcy

In [21]:
correct_gen = (pred[1].argmax(1) == gen).type(torch.float).sum().item()
correct_eth = (pred[2].argmax(1) == eth).type(torch.float).sum().item()

total = len(y)

In [22]:
print(total)

64


In [18]:
print(correct_gen)

28.0


In [19]:
print(correct_eth)

18.0


In [20]:
print(pred[1].shape)

torch.Size([64, 2])


In [23]:
print(correct_gen/total)

0.4375


In [25]:
print(len(test_loader.dataset))

4741
